In [17]:
# %pip install llama-index-readers-file
# %pip install --u llama-index
# %pip install llama-parse
# %pip install openai
# %pip install langchain
# %pip install --upgrade numpy
# %pip install tiktoken
# %pip install python-dotenv
# %pip install -U sentence-transformers
# %pip install langchain_experimental
# %pip install langchain_openai
# %pip install --upgrade langchain_openai langchain_core


In [18]:
import os
import openai
from dotenv import load_dotenv
from llama_index.core import SimpleDirectoryReader, Document
from llama_parse import LlamaParse  # pip install llama-parse

import nest_asyncio
nest_asyncio.apply()

# import param
# from huggingface_hub import login

# Loading Environment variables:
dotenv_path = 'KEYs.env'  
_ = load_dotenv(dotenv_path)

openai.api_key = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY=os.environ['PINECONE_API_KEY']
LLAMA_CLOUD_API_KEY=os.environ['LLAMA_CLOUD_API_KEY']
# hf_token = os.environ['hf_token']
# login(token=hf_token)

In [19]:
# Read document

parser = LlamaParse(
    api_key=LLAMA_CLOUD_API_KEY,
    result_type="markdown"  # "markdown" and "text" are available
    #verbose=True
)

def get_meta(file_path):
    return {"source": file_path}

file_extractor = {".pdf": parser}
reader = SimpleDirectoryReader(input_dir="C:/Users/marlo/OneDrive/Desktop/Anaconda/Fun/Deep_Learning/Semantic_Search_Project/Database",
                               #input_files=["013-028l_S3_Klassifikation-Diagnostik-Therapie-Urtikaria_2022-04.pdf"],
                                recursive=True, 
                                num_files_limit=3,
                                file_extractor=file_extractor,
                                filename_as_id=True,
                                file_metadata=get_meta
                                )
docs = reader.load_data()

print(type(docs), "\n")
print(len(docs), "\n")
print(type(docs[0]))
print(type(docs[0].text))

Started parsing the file under job_id ab32abf1-db4b-4923-828e-b47f628dd740
Started parsing the file under job_id 6d89467c-b7c5-4f31-a2e7-5d6bbad07da4
Started parsing the file under job_id de9728b0-7f04-425f-9bae-8495148e9d46
<class 'list'> 

3 

<class 'llama_index.core.schema.Document'>
<class 'str'>


In [43]:
for i in range(len(docs)):
    print(i)
    print(docs[i].metadata['source'])
#print(documents[0].text)

0
C:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\Database\__AWMF-Regelwerk\20210215_AWMF-Regelwerk_2020_V2.0.pdf
1
C:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\Database\Adipositas\050-001l_S3_Adipositas_Prävention_Therapie_2014-11-abgelaufen.pdf
2
C:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\Database\Adipositas\050-002l_S3_Therapie-Praevention-Adipositas-Kinder-Jugendliche_2019-11.pdf


### 2. Text Normalization

In [28]:
# Normalize Text
import re
# List of expressions to replace
expressions_to_replace = [
    "bzw\.", "z\. B\.", "med\.", "Dr\.", "zit\.", "n\.", "e.V\.", "rer\.", "nat\.", "Prof\.", "B.A\.",
    "Ca\.", "ca\.", "usw\.", "v\. a\.", "p\. p\.", "s\. o\.", "s\. u\.", "sog\.", "u\. a\.", "vs\.",
    "Min\.", "et al\.", "ärztl\.", "evtl\.", "ggf\."
]

for i in range(len(docs)):
    print("iteration:",i)
    # Replace the expressions in the markdown text
    for expression in expressions_to_replace:
        docs[i].text = re.sub(expression, expression.replace("\\.", ""), docs[i].text)

iteration: 0
iteration: 1
iteration: 2


### 3. Extract Page Number Metadata

In [30]:
# '''
# Method: "Unsexy but works"
# 1. Read Markdown Text. Whenever the page delimiter (i.e. "---") is encountered, write the current page count above the line (e.g. (Page x)) (Note: Llamaparse reliably detects all page breaks)
# 2. For each Header (marked by ###,##,#) write page number into the Header
# 3. Let MarkdownHeaderTextSplitter split the text (During Chunking). This will automatically create chunks with metadata containing "Header" information.
# 4. Now we extract the Page number previously written into the Header, and write it as a new entry in the metadata dictionary under "Page Number"
# 5. Delete all the (Page x) added previously to the headers and page_content to restore our original markdown text.

# '''

# import re

# # 1. Read Markdown Text. Whenever the page delimiter (i.e. "---") is encountered, write the current page count above the line (e.g. (Page x))

# def count_and_update_delimiters(markdown_text):
#     # Compile regex pattern to match "---" delimiters on separate lines
#     delimiter_pattern = re.compile(r'^\s*---\s*$', re.MULTILINE)

#     # Initialize count
#     delimiter_count = 0

#     # Function to replace each "---" delimiter with "--- {count} delimiter(s) found"
#     def replace(match):
#         nonlocal delimiter_count
#         delimiter_count += 1
#         return f'\n(Page {delimiter_count})\n---\n'

#     # Replace each "---" delimiter
#     updated_markdown_text = re.sub(delimiter_pattern, replace, markdown_text)

#     return updated_markdown_text

# # 2. For each Header (marked by ###,##,#) write page number into the Header
# def modify_headers_with_page_numbers(markdown_text):
#     # Compile regex pattern to match headers (###, ##, #) and "page_nr: x"
#     header_pattern = re.compile(r'^(#+)\s+(.*)$', re.MULTILINE)
#     page_number_pattern = re.compile(r'(Page \s*(\d+))', re.IGNORECASE)

#     # Function to replace each header with header + page number
#     def replace(match):
#         header_level = match.group(1)
#         header_text = match.group(2)
#         page_nr_match = page_number_pattern.search(markdown_text, match.end())
#         if page_nr_match:
#             page_nr = page_nr_match.group(1)
#             return f'{header_level} {header_text} ({page_nr})'
#         else:
#             return match.group(0)

#     # Replace each header
#     updated_markdown_text = re.sub(header_pattern, replace, markdown_text)

#     return updated_markdown_text


# updated_text = count_and_update_delimiters(docs[1].text)
# modified_text = modify_headers_with_page_numbers(updated_text)
# print(modified_text)



## Interdisziplinäre Leitlinie der Qualität S3 zur „Prävention und Therapie der Adipositas“ (Page 1)

|AWMF-Register Nr.|050/001|Klasse:|S3|
|---|---|---|---|
|Version|2.0 (April 2014)| | |

DEUTSCHE ADIPOSITAS GESELLSCHAFT

Deutsche Diabetes Gesellschaft

DGE

Deutsche Gesellschaft für Ernährung e.V

DGEM

Deutsche Gesellschaft für Ernährungsmedizin e.V

(Page 1)
---

## Impressum (Page 2)

Herausgeber

|1. Auflage (Ersterstellung, 2007):|Deutsche Adipositas-Gesellschaft (DAG) e.V, Deutsche Diabetes Gesellschaft (DDG), Deutsche Gesellschaft für Ernährung (DGE) e.V, Deutsche Gesellschaft für Ernährungsmedizin (DGEM) e.V|
|---|---|
|2. Auflage (1. Aktualisierung, 2011-2014):|Deutsche Adipositas-Gesellschaft (DAG) e.V, Deutsche Diabetes Gesellschaft (DDG), Deutsche Gesellschaft für Ernährung (DGE) e.V, Deutsche Gesellschaft für Ernährungsmedizin (DGEM) e.V|

Federführende Fachgesellschaft

Deutsche Adipositas-Gesellschaft (DAG) e.V
(Präsident: Prof Dr M. Wabitsch)

Kontakt

Deutsche Adip

### 4. Chunking Algorithm

In [56]:
# NEW Chunker Alg
'''
Method:
1. Markdown Chunking: First use the MarkdownTextSplitter to do “rule-based chunking” using the titles and headers.
2. Semantic Chunking:
    2.1. Check the chunk size of the resulting chunks
    2.2. If chunk size is below min token threshold (i.e. 20 tokens) --> merge the chunk with the previous chunk
    2.3. If chunk size exceeds the max chunk size of the embedding model (i.e. 512 for e5-multilingual) --> subchunk that chunk with the SemanticChunker.
        2.3.1. Find the chunks with token size above max token limit (512 tokens)
        2.3.2. Split the chunks into subchunks using breakpoint value of 95 (to begin with)
        2.3.3. Check subchunks:
            2.3.3.1 if the created subchunks are smaller than min token limit --> merge with previous subchunk
            2.3.3.2 Substitute the original "parent chunk" with its subchunks (Note: Metadata of parent chunks are copied to children)
        4. Iteratively check again the size each resulting subchunk. (go back to 2.3.1)
        5. If the subchunks cannot be broken down any further, (i.e. SemanticChunker is unable to break at threshold 95) --> reduce threshold to 85

'''
'''
Improvements:
- Whenever there is a header, a new chunk must start. i.e. cannot have header in the middle of chunk
- Embedding similarity idea also for merging when below min limit.
'''

# MarkdownTextSplitter
from langchain.text_splitter import MarkdownHeaderTextSplitter
import tiktoken
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings
from tqdm import tqdm
import re

# helper functions

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    #num_tokens = len(encoding.encode(string, disallowed_special = set()))

    return num_tokens

def semantic_chunker(threshold_amount):
    """Creates an instance of a SemanticChunker. Input threshold value for breakpoint splitting"""
    text_splitter = SemanticChunker(
        OpenAIEmbeddings(),
        breakpoint_threshold_type="percentile",
        #breakpoint_threshold_type="standard_deviation"
        #breakpoint_threshold_type="interquartile"
        breakpoint_threshold_amount = threshold_amount
    )
    return text_splitter

def print_all_chunks(list_of_docs,with_page_content=False, with_num_tokens=True):

    for i in range(len(list_of_docs)):
        num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")
        if with_num_tokens == True:
            print(f"num. of tokens in chunk {i} is: {num_of_tokens}")
        if with_page_content == True:
            #print(f"-----------------------------CHUNK {i} ----------------------------------")
            #print(list_of_docs[i].page_content)
            print(f"---")
            print(list_of_docs[i].page_content)

def find_idxs_below_min_chunk_size(list_of_docs, min_chunk_size=20, show_indices=True):

    idxs_below_min_chunk_size = []

    for i in range(len(list_of_docs)):
        num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")

        # if chunk size is below min_chunk_size, append index to list
        if num_of_tokens < min_chunk_size:
            idxs_below_min_chunk_size.append(i)

    num_of_idxs_in_list = len(idxs_below_min_chunk_size)

    if show_indices==True:
        print(f"{num_of_idxs_in_list} indices below min threshold: {idxs_below_min_chunk_size}")

    return idxs_below_min_chunk_size

def find_idxs_above_max_chunk_size(list_of_docs, max_chunk_size=512):

    idxs_above_max_chunk_size = []

    for i in range(len(list_of_docs)):
        num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")

        # if chunk size is below min_chunk_size, append index to list
        if num_of_tokens > max_chunk_size:
            idxs_above_max_chunk_size.append(i)

    num_of_idxs_in_list = len(idxs_above_max_chunk_size)

    print(f"{num_of_idxs_in_list} indices above max threshold: {idxs_above_max_chunk_size}")

    return idxs_above_max_chunk_size

def remove_below_min_chunks_list(list_of_docs, list_blw_min_idx, min_chunk_size=20):

    # inverse list order to not get indexing problems
    sorted_max_indices = sorted(list_blw_min_idx, reverse=True)
    #print(sorted_max_indices)

    for m in sorted_max_indices:

        num_of_tokens = num_tokens_from_string(list_of_docs[m].page_content, "cl100k_base")

        if num_of_tokens < min_chunk_size and m != 0:
            update_data = {
                "page_content": list_of_docs[m-1].page_content + " \n" + list_of_docs[m].page_content,
            }
            new_doc_node = list_of_docs[m-1].copy(update = update_data)

            # delete the two old nodes and subsitute it with the new node
            list_of_docs.pop(m)
            list_of_docs.pop(m-1)

            #insert new merged node into list
            list_of_docs.insert(m-1,new_doc_node)

def list_of_str_2_list_of_docs(list_of_str, doc_obj):

    list_of_docs = []

    for n in range(len(list_of_str)):

        data = {
            "page_content": list_of_str[n]
        }
        new_doc_obj = doc_obj.copy(update = data)
        list_of_docs.append(new_doc_obj)

    #print(list_of_docs)
    return list_of_docs

def remove_above_max_chunks_list(list_of_docs, list_abv_max_idx, breakpoint_thresh_value=95):

    # inverse list order to not get indexing problems
    sorted_max_indices = sorted(list_abv_max_idx, reverse=True)
    #print(sorted_max_indices)

    for m in tqdm(sorted_max_indices):

            # semantically split chunk
            subchunks = semantic_chunker(breakpoint_thresh_value).split_text(list_of_docs[m].page_content)

            # right now subchunks is a list of str (i.e. each element only contains text).
            # convert this list of strings into a list of docs
            node = list_of_docs[m].copy() # create copy of parent node (needed for list_of_str_2_list_of_docs fct.)
            docs = list_of_str_2_list_of_docs(subchunks, node)

            # Check, that the created subchunks are of token size > min_chunk_size (=20) --> if not, merge them with previous subchunk
            subchunk_idxs_below_min_thresh = find_idxs_below_min_chunk_size(docs, min_chunk_size=100, show_indices=False)
            remove_below_min_chunks_list(docs, subchunk_idxs_below_min_thresh, min_chunk_size=100)

            num_of_subchunks=len(docs)

            #create copy of the node
            copy_node_2 = list_of_docs[m].copy()
            #print(copy_node)

            # delete node from list
            list_of_docs.pop(m)

            for n in reversed(range(num_of_subchunks)):

                update_data = {
                    "page_content": docs[n].page_content
                }
                copy_node_2_updated = copy_node_2.copy(update = update_data)

                #insert new merged node into list
                list_of_docs.insert(m, copy_node_2_updated)

def write_results_in_txt(list_of_docs,leitlinien_doc,list_of_above_max_indices, output_file):
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(f"-----------------------------RESULTS ----------------------------------\n")
        file.write(f"Leitlinien doc: {leitlinien_doc} \n")
        file.write(f"final num. of chunks: {len(list_of_docs)}\n")
        file.write(f"chunks over max. token limit: {list_of_above_max_indices}\n")

        for i in range(len(list_of_docs)):
            num_of_tokens = num_tokens_from_string(list_of_docs[i].page_content, "cl100k_base")
            file.write(f"num. of tokens in chunk {i} is: {num_of_tokens} \n")
            file.write(f"-----------------------------CHUNK {i} ----------------------------------\n")
            file.write(list_of_docs[i].page_content + '\n')

# Page number functions
'''
Method: "Unsexy but works"
1. Read Markdown Text. Whenever the page delimiter (i.e. "---") is encountered, write the current page count above the line (e.g. (Page x)) (Note: Llamaparse reliably detects all page breaks)
2. For each Header (marked by ###,##,#) write page number into the Header
3. Let MarkdownHeaderTextSplitter split the text (During Chunking). This will automatically create chunks with metadata containing "Header" information.
4. Now we extract the Page number previously written into the Header, and write it as a new entry in the metadata dictionary under "Page Number"
5. Delete all the (Page x) added previously to the headers and page_content to restore our original markdown text.

'''

def count_and_update_delimiters(markdown_text):
    # Compile regex pattern to match "---" delimiters on separate lines
    delimiter_pattern = re.compile(r'^\s*---\s*$', re.MULTILINE)

    # Initialize count
    delimiter_count = 0

    # Function to replace each "---" delimiter with "--- {count} delimiter(s) found"
    def replace(match):
        nonlocal delimiter_count
        delimiter_count += 1
        return f'\n(Page {delimiter_count})\n---\n'

    # Replace each "---" delimiter
    updated_markdown_text = re.sub(delimiter_pattern, replace, markdown_text)

    return updated_markdown_text

def modify_headers_with_page_numbers(markdown_text):
    # Compile regex pattern to match headers (###, ##, #) and "page_nr: x"
    header_pattern = re.compile(r'^(#+)\s+(.*)$', re.MULTILINE)
    page_number_pattern = re.compile(r'(Page \s*(\d+))', re.IGNORECASE)

    # Function to replace each header with header + page number
    def replace(match):
        header_level = match.group(1)
        header_text = match.group(2)
        page_nr_match = page_number_pattern.search(markdown_text, match.end())
        if page_nr_match:
            page_nr = page_nr_match.group(1)
            return f'{header_level} {header_text} ({page_nr})'
        else:
            return match.group(0)

    # Replace each header
    updated_markdown_text = re.sub(header_pattern, replace, markdown_text)

    return updated_markdown_text

def update_metadata_with_page_numbers(md_header_splits):
    '''
    This function:
    1. reads the page numbers, previously inserted into the headers, and writes them into a seperate metadata entry: Page Number
    2. deletes all the page numbers inserted previously into the header metadata entries
    '''
    # Compile regex pattern to extract page numbers from headers
    page_number_pattern = re.compile(r'\(Page\s+(\d+)\)')

    for header_split in md_header_splits:
        metadata = header_split.metadata
        page_content = header_split.page_content
        page_numbers = []

        if metadata:
            last_header_value = list(metadata.values())[-1]
            page_number_match = page_number_pattern.search(last_header_value)
            if page_number_match:
                page_number = int(page_number_match.group(1))
                page_numbers.append(page_number)
                metadata['Page Number'] = page_numbers
            for key, value in metadata.items():
                if re.search(r'\bHeader\b', key):
                  # Remove the page number and surrounding brackets from the header
                  value_without_page_number = re.sub(page_number_pattern, '', value).strip()

                  # Update the metadata dictionary
                  metadata[key] = value_without_page_number
                if key == 'Page Number':
                    page_nums = set(int(match.group(1)) for match in page_number_pattern.finditer(page_content))
                    for num in page_nums:
                        if num not in page_numbers:
                            page_numbers.append(num)
                            metadata['Page Number'] = page_numbers

def update_page_content(md_header_splits):
    '''
    This function deletes all the page numbers (i.e. (Page x)) inserted previously into the page content
    '''
    # Compile regex pattern to match (Page x)
    page_number_pattern = re.compile(r'\(Page\s+\d+\)')

    for header_split in md_header_splits:
        page_content = header_split.page_content
        # Remove (Page x) from the page content
        updated_page_content = re.sub(page_number_pattern, '', page_content)
        # Update the page_content in md_header_splits
        header_split.page_content = updated_page_content

def update_metadata_with_source(md_header_splits,documents):
    '''
    This function includes the document source into the final chunks metadata
    '''
    for header_split in md_header_splits:
        header_split.metadata['source'] = documents.metadata['source']

# 1. MarkdownTextSplitter

headers_to_split_on = [
    ("#", "Header 1"),
    ("##", "Header 2"),
    ("###", "Header 3"),
]

markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on, strip_headers=False)

# Settings
min_chunk_size = 300
max_chunk_size = 2000 #ada: 8191, e-5: 512

# List containing all chunks
doc_chunks = []


d = 0
for d in range(len(docs)):

    documents = docs[d]
    print("documents TEST MARLON",d)
    #print("documents TEST MARLON",docs[0].metadata)
    print("documents TEST MARLON",documents.metadata['source'])
    #print("documents TEST MARLON",documents.id)
    print("documents TEST MARLON",len(docs))

    # add page number data into markdown
    updated_text = count_and_update_delimiters(documents.text)
    modified_text = modify_headers_with_page_numbers(updated_text)

    # Perform Markdown Text Splitting
    md_header_splits = markdown_splitter.split_text(modified_text)
    #md_header_splits = markdown_splitter.split_text(documents[0].text)

    # add page number metadata
    update_metadata_with_page_numbers(md_header_splits)
    #md_header_splits.metadata['source'] = documents.metadata['source']
    update_metadata_with_source(md_header_splits,documents)
    update_page_content(md_header_splits)
    #print("documents TEST MARLON md_header_splits",md_header_splits)
    print("documents TEST MARLON md_header_splits",len(md_header_splits))
    print("documents TEST MARLON md_header_splits",md_header_splits[0])
    # Print the Chunks extracted by MarkdownHeaderTextSplitter:
    #print_all_chunks(md_header_splits)

    print("----------------------------------MarkdownHeaderTextSplitter----------------------------------")

    print(f"num. of chunks after MarkdownHeaderTextSplitter: {len(md_header_splits)}")

    for i in range(len(md_header_splits)):
        str = num_tokens_from_string(md_header_splits[i].page_content, "cl100k_base")
        print(f"num. of tokens in chunk {i} is: {str}")
        print(md_header_splits[i].metadata)

    print("Indices that are below min/above max token size: ")
    list_of_below_min_indices = find_idxs_below_min_chunk_size(md_header_splits, min_chunk_size)
    list_of_above_max_indices= find_idxs_above_max_chunk_size(md_header_splits, max_chunk_size)
    i = 0
    #num_of_idxs_in_list = []
    previous_indices = None
    breakpoint_thresh_value = 95

    print("----------------------------------MIN_MAX_SUBCHUNKING:----------------------------------")
    while len(list_of_below_min_indices) > 0 or len(list_of_above_max_indices) > 0:

        list_of_below_min_indices = find_idxs_below_min_chunk_size(md_header_splits, min_chunk_size)

        if len(list_of_below_min_indices) > 0:
            remove_below_min_chunks_list(md_header_splits, list_of_below_min_indices, min_chunk_size)
            print(f"-----------------------CHUNKS AFTER MIN REMOVAL {i}--------------------------")
            print(f"num. of chunks after {i} MIN_SUBCHUNKING: {len(md_header_splits)}")
            #print_all_chunks(md_header_splits, with_page_content=False)

        list_of_above_max_indices= find_idxs_above_max_chunk_size(md_header_splits, max_chunk_size)

        # Check, if two consecutive numbers in the num_of_idxs_in_list = [] are the same, decrease the breakpoint threshold by 10.
        if previous_indices is not None and list_of_above_max_indices == previous_indices and breakpoint_thresh_value!=75:
            breakpoint_thresh_value=breakpoint_thresh_value-10

        if list_of_above_max_indices == previous_indices and breakpoint_thresh_value==75:
            print("Finished!")
            break

        # Update the previous indices for the next iteration
        previous_indices = list_of_above_max_indices

        if len(list_of_above_max_indices) > 0:
            remove_above_max_chunks_list(md_header_splits, list_of_above_max_indices, breakpoint_thresh_value)
            print(f"-----------------------CHUNKS AFTER MAX REMOVAL {i}--------------------------")
            print(f"num. of chunks after {i} MAX_SUBCHUNKING: {len(md_header_splits)}")
            #print_all_chunks(md_header_splits, with_page_content=False)

        #print(md_header_splits)
        print(len(md_header_splits))

        i = i+1

    print("-----------------------FINAL CHUNKS--------------------------")

    doc_chunks.append(md_header_splits)
    print_all_chunks(md_header_splits, with_page_content=False)

    leitlinien_doc= documents.id_
    output_file = f"chunking_results_{d}.txt"
    open(output_file, 'w').close()
    write_results_in_txt(md_header_splits,leitlinien_doc,list_of_above_max_indices, output_file)


documents TEST MARLON 0
documents TEST MARLON C:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\Database\__AWMF-Regelwerk\20210215_AWMF-Regelwerk_2020_V2.0.pdf
documents TEST MARLON 3
documents TEST MARLON md_header_splits 115
documents TEST MARLON md_header_splits page_content='# $AWMF   \nArbeitsgemeinschaft der Wissenschaftlichen Medizinischen Fachgesellschaften e.V  \nDas AWMF-Regelwerk Leitlinien Version 2.0  \nHerausgebende:  \nStändige Kommission „Leitlinien“ der Arbeitsgemeinschaft der Wissenschaftlichen Medizinischen Fachgesellschaften (AWMF)  \n© AWMF 2020  \n\n---  \nRedaktion /Kontakt:  \nAWMF-Institut für Medizinisches Wissensmanagement (AWMF-IMWi)\nc/o Philipps-Universität\nKarl-von-Frisch-Str. 1\n35043 Marburg\nE-Mail: imwi@awmf.org\nKommentare und Änderungsvorschläge bitte an diese Adresse  \nStändige Kommission Leitlinien der AWMF\nSpies C, Kopp I, Nothacker M, Barzel A, Beckmann M, Diel F, Follmann M, Gaebel W, Jonitz G, Klemperer D, K

100%|██████████| 5/5 [00:06<00:00,  1.36s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 0--------------------------
num. of chunks after 0 MAX_SUBCHUNKING: 87
87
4 indices below min threshold: [9, 33, 56, 65]
-----------------------CHUNKS AFTER MIN REMOVAL 1--------------------------
num. of chunks after 1 MIN_SUBCHUNKING: 83
0 indices above max threshold: []
83
0 indices below min threshold: []
0 indices above max threshold: []
83
-----------------------FINAL CHUNKS--------------------------
num. of tokens in chunk 0 is: 1589
num. of tokens in chunk 1 is: 717
num. of tokens in chunk 2 is: 462
num. of tokens in chunk 3 is: 1021
num. of tokens in chunk 4 is: 782
num. of tokens in chunk 5 is: 623
num. of tokens in chunk 6 is: 792
num. of tokens in chunk 7 is: 301
num. of tokens in chunk 8 is: 700
num. of tokens in chunk 9 is: 1363
num. of tokens in chunk 10 is: 509
num. of tokens in chunk 11 is: 549
num. of tokens in chunk 12 is: 784
num. of tokens in chunk 13 is: 996
num. of tokens in chunk 14 is: 500
num. of tokens in chunk 

100%|██████████| 13/13 [00:22<00:00,  1.70s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 0--------------------------
num. of chunks after 0 MAX_SUBCHUNKING: 117
117
26 indices below min threshold: [0, 18, 44, 46, 49, 59, 66, 67, 68, 77, 78, 79, 80, 93, 95, 96, 99, 100, 101, 102, 103, 105, 107, 108, 109, 110]
-----------------------CHUNKS AFTER MIN REMOVAL 1--------------------------
num. of chunks after 1 MIN_SUBCHUNKING: 99
6 indices above max threshold: [39, 41, 47, 50, 53, 90]


...

-----------------------CHUNKS AFTER MAX REMOVAL 1--------------------------
num. of chunks after 1 MAX_SUBCHUNKING: 109
109
2 indices below min threshold: [0, 60]
-----------------------CHUNKS AFTER MIN REMOVAL 2--------------------------
num. of chunks after 2 MIN_SUBCHUNKING: 108
2 indices above max threshold: [39, 42]


100%|██████████| 2/2 [00:02<00:00,  1.44s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 2--------------------------
num. of chunks after 2 MAX_SUBCHUNKING: 110
110
1 indices below min threshold: [0]
-----------------------CHUNKS AFTER MIN REMOVAL 3--------------------------
num. of chunks after 3 MIN_SUBCHUNKING: 110
0 indices above max threshold: []
110
1 indices below min threshold: [0]
-----------------------CHUNKS AFTER MIN REMOVAL 4--------------------------
num. of chunks after 4 MIN_SUBCHUNKING: 110
0 indices above max threshold: []
110
1 indices below min threshold: [0]
-----------------------CHUNKS AFTER MIN REMOVAL 5--------------------------
num. of chunks after 5 MIN_SUBCHUNKING: 110
0 indices above max threshold: []
Finished!
-----------------------FINAL CHUNKS--------------------------
num. of tokens in chunk 0 is: 127
num. of tokens in chunk 1 is: 1472
num. of tokens in chunk 2 is: 300
num. of tokens in chunk 3 is: 1735
num. of tokens in chunk 4 is: 541
num. of tokens in chunk 5 is: 486
num. of tokens in chunk

100%|██████████| 4/4 [00:08<00:00,  2.06s/it]


-----------------------CHUNKS AFTER MAX REMOVAL 0--------------------------
num. of chunks after 0 MAX_SUBCHUNKING: 104
104
11 indices below min threshold: [18, 21, 27, 29, 38, 39, 41, 42, 47, 48, 50]
-----------------------CHUNKS AFTER MIN REMOVAL 1--------------------------
num. of chunks after 1 MIN_SUBCHUNKING: 95
0 indices above max threshold: []
95
0 indices below min threshold: []
0 indices above max threshold: []
95
-----------------------FINAL CHUNKS--------------------------
num. of tokens in chunk 0 is: 464
num. of tokens in chunk 1 is: 611
num. of tokens in chunk 2 is: 1020
num. of tokens in chunk 3 is: 1146
num. of tokens in chunk 4 is: 662
num. of tokens in chunk 5 is: 346
num. of tokens in chunk 6 is: 311
num. of tokens in chunk 7 is: 506
num. of tokens in chunk 8 is: 1009
num. of tokens in chunk 9 is: 846
num. of tokens in chunk 10 is: 858
num. of tokens in chunk 11 is: 654
num. of tokens in chunk 12 is: 481
num. of tokens in chunk 13 is: 651
num. of tokens in chunk 14 

In [57]:
print(md_header_splits[0])
print(len(md_header_splits))

page_content='## S3-Leitlinie Therapie und Prävention der Adipositas im Kindes- und Jugendalter   \nARBEITSGEMEINSCHAFT der Deutschen Adipositas-Gesellschaft ADIPOSITAS IM KINDES- UND JUGENDALTER Korporatives Mitglied der Deutschen Gesellschaft fur Kinder- und Jugendmedizin  \nEvidenzbasierte (S3-) Leitlinie der Arbeitsgemeinschaft Adipositas im Kindes- und Jugendalter (AGA) der Deutschen Adipositas-Gesellschaft (DAG) und der Deutschen Gesellschaft für Kinder-und Jugendmedizin (DGKJ)  \nTherapie und Prävention der Adipositas im Kindes- und Jugendalter AWMF-Nr. 050-002  \nin Zusammenarbeit mit: Version August 2019 \n### Fachgesellschaften   \nDeutsche Diabetes Gesellschaft (DDG)\nDeutsche Gesellschaft für Ernährung (DGE)\nDeutsche Gesellschaft für Ernährungsmedizin (DGEM)\nDeutsche Gesellschaft für Kinder- und Jugendpsychiatrie, Psychosomatik und Psychoperapie (DGKJP)\nDeutsche Gesellschaft für Kinderendokrinologie und -diabetologie (DGKED)\nDeutsche Gesellschaft für Rehabilitationswiss

### Metadata Modification

In [58]:
for i, split in enumerate(md_header_splits):
    # split.metadata['Page Number'][0] +=1
    if "abgelaufen" not in split.metadata["source"]:
        split.metadata["Gültigkeit"] = "Gültig"
    else:
        split.metadata["Gültigkeit"] = "Abgelaufen"

### Pincone Loading

In [59]:
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer

pinecone = Pinecone(api_key=PINECONE_API_KEY)

INDEX_NAME = 'pinecone-test'
index = pinecone.Index(INDEX_NAME)
index.describe_index_stats()

model = SentenceTransformer("BAAI/bge-small-en-v1.5") # 'all-MiniLM-L6-v2'

c:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\streamlit\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [85]:
# Preprocessing Data Load for splits
vector_limit=10

splits = md_header_splits[:vector_limit]
splits_metadata_source = []
splits_metadata_page = []
splits_metadata_Gültigkeit = []
splits_page_content = []
#preprocess
for i in range(vector_limit):
    splits_page_content.append(splits[i].page_content)
    metadata = splits[i].metadata
    print(metadata)
    print(metadata["source"])
    splits_metadata_source.append(metadata['source'])
    splits_metadata_page.append(metadata['Page Number'][0])
    splits_metadata_Gültigkeit.append(metadata['Gültigkeit'])
    #splits_metadata.append(splits[i].metadata)


print(splits_metadata_source)
print(len(splits_metadata_source))

print(splits_metadata_page)
print(len(splits_metadata_page))

print(splits_page_content)
print(len(splits_page_content))

{'Header 2': 'S3-Leitlinie Therapie und Prävention der Adipositas im Kindes- und Jugendalter', 'Page Number': [2], 'source': 'C:\\Users\\marlo\\OneDrive\\Desktop\\Anaconda\\Fun\\Deep_Learning\\Semantic_Search_Project\\Database\\Adipositas\\050-002l_S3_Therapie-Praevention-Adipositas-Kinder-Jugendliche_2019-11.pdf', 'Gültigkeit': 'Gültig'}
C:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\Database\Adipositas\050-002l_S3_Therapie-Praevention-Adipositas-Kinder-Jugendliche_2019-11.pdf
{'Header 2': 'S3-Leitlinie Therapie und Prävention der Adipositas im Kindes- und Jugendalter', 'Page Number': [3], 'source': 'C:\\Users\\marlo\\OneDrive\\Desktop\\Anaconda\\Fun\\Deep_Learning\\Semantic_Search_Project\\Database\\Adipositas\\050-002l_S3_Therapie-Praevention-Adipositas-Kinder-Jugendliche_2019-11.pdf', 'Gültigkeit': 'Gültig'}
C:\Users\marlo\OneDrive\Desktop\Anaconda\Fun\Deep_Learning\Semantic_Search_Project\Database\Adipositas\050-002l_S3_Therapie-Praevention-Adip

In [89]:
# Upserting Data to pinecone
# from tqdm.auto import tqdm

batch_size=5

# splits = splits[:vector_limit]

for i in tqdm(range(0, len(splits), batch_size)):
    # find end of batch
    i_end = min(i+batch_size, len(splits))
    # create IDs batch
    print(x for x in range(i, i_end))
    ids = [str(x) for x in range(i, i_end)]
    # create metadata batch
    metadatas = [
                    {'text': text,'source': source, 'page': page, 'Gültigkeit': gültigkeit}
                    for text, source, page, gültigkeit in zip(splits_page_content[i:i_end], splits_metadata_source[i:i_end], splits_metadata_page[i:i_end], splits_metadata_Gültigkeit[i:i_end])
                ]

    # create embeddings
    print(splits_page_content[i:i_end])
    xc = model.encode(splits_page_content[i:i_end])
    print(type(xc.tolist()))
    xc = xc.tolist()
    # create records list for upsert
    records = zip(ids, xc, metadatas)
    # upsert to Pinecone
    index.upsert(vectors=records)

  0%|          | 0/2 [00:00<?, ?it/s]

0
<generator object <genexpr> at 0x00000206CE49CD60>
['## S3-Leitlinie Therapie und Prävention der Adipositas im Kindes- und Jugendalter   \nARBEITSGEMEINSCHAFT der Deutschen Adipositas-Gesellschaft ADIPOSITAS IM KINDES- UND JUGENDALTER Korporatives Mitglied der Deutschen Gesellschaft fur Kinder- und Jugendmedizin  \nEvidenzbasierte (S3-) Leitlinie der Arbeitsgemeinschaft Adipositas im Kindes- und Jugendalter (AGA) der Deutschen Adipositas-Gesellschaft (DAG) und der Deutschen Gesellschaft für Kinder-und Jugendmedizin (DGKJ)  \nTherapie und Prävention der Adipositas im Kindes- und Jugendalter AWMF-Nr. 050-002  \nin Zusammenarbeit mit: Version August 2019 \n### Fachgesellschaften   \nDeutsche Diabetes Gesellschaft (DDG)\nDeutsche Gesellschaft für Ernährung (DGE)\nDeutsche Gesellschaft für Ernährungsmedizin (DGEM)\nDeutsche Gesellschaft für Kinder- und Jugendpsychiatrie, Psychosomatik und Psychoperapie (DGKJP)\nDeutsche Gesellschaft für Kinderendokrinologie und -diabetologie (DGKED)\nDeut

  0%|          | 0/2 [00:01<?, ?it/s]

<class 'list'>


TypeError: 'int' object is not iterable